In [199]:
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [200]:
data = pd.read_csv("./Data/test.csv",sep=",",header=0)

In [201]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2553,90,RL,74,6882,Pave,NAW,Reg,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,Duplex,1Story,4,3,1955,1955,Gable,CompShg,AsbShng,Plywood,BrkCmn,128,TA,TA,PConc,NB,NB,NB,NB,0,NB,0,0,0,GasA,TA,N,SBrkr,1152,0,0,1152,0,0,2,0,2,2,Fa,6,Typ,0,NF,NG,NG,NG,0,0,NG,NG,N,0,0,0,0,0,0,NP,NF,Nfeat,0,9,2007,WD,Normal
1,2141,85,RL,67,7703,Pave,NAW,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,SFoyer,6,8,1978,1978,Gable,CompShg,HdBoard,HdBoard,BrkCmn,40,TA,TA,CBlock,Gd,TA,Gd,ALQ,450,Unf,0,0,450,GasA,Ex,Y,SBrkr,1034,0,0,1034,0,1,1,0,3,1,TA,6,Typ,1,Po,Basment,1978,Fin,2,504,TA,TA,Y,311,0,0,0,0,0,NP,GdWo,Nfeat,0,5,2008,WD,Normal
2,2260,20,RL,160,18160,Grvl,NAW,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,6,6,1964,1964,Gable,CompShg,HdBoard,HdBoard,BrkCmn,138,TA,TA,CBlock,TA,TA,Av,BLQ,550,Unf,0,752,1302,GasA,Fa,Y,SBrkr,1128,0,0,1128,1,0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1964,Unf,2,480,TA,TA,P,0,108,246,0,0,0,NP,MnPrv,Nfeat,0,3,2007,WD,Alloca
3,2762,20,RL,77,9510,Pave,NAW,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1962,1985,Gable,CompShg,HdBoard,HdBoard,BrkCmn,161,TA,TA,CBlock,TA,TA,No,ALQ,701,Unf,0,434,1135,GasA,Ex,Y,SBrkr,1207,0,0,1207,0,0,1,1,3,1,TA,6,Typ,0,NF,Attchd,1962,RFn,1,264,TA,TA,Y,0,240,0,0,0,0,NP,NF,Nfeat,0,5,2006,WD,Normal
4,2409,20,RL,70,9100,Pave,NAW,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1960,1960,Hip,CompShg,HdBoard,HdBoard,BrkCmn,69,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,1144,1144,GasA,TA,Y,SBrkr,1144,0,0,1144,0,0,1,0,3,1,TA,6,Typ,1,Gd,Attchd,1960,RFn,1,336,TA,TA,Y,0,0,0,0,0,0,NP,NF,Nfeat,0,6,2007,WD,Normal


In [202]:
X_all = data.ix[:,:]
print("Explanatory Variables: {}".format(X_all.shape[1]))
#y = data.ix[:,-1:]
#print("Response Variable: {}".format(y.shape[1]))
#print("Total Observations: {}".format(y.shape[0]))

Explanatory Variables: 80


In [203]:
X_all.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.00000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.334476,9819.161069,6.078821,5.553804,1971.357779,1983.662783,99.673749,438.902673,52.583276,553.915010,1045.400960,1156.534613,325.967786,3.543523,1486.045922,0.435230,0.066484,1.570939,0.377656,2.854010,1.042495,6.385195,0.58122,1.766278,472.856066,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,20.569211,4955.517327,1.436812,1.113740,30.390071,21.130467,177.001792,455.257119,176.698671,437.351324,443.592976,398.165820,420.610226,44.043251,485.566099,0.530696,0.254656,0.555190,0.503017,0.829788,0.208472,1.508895,0.64742,0.775703,216.999731,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,407.000000,0.000000,0.000000,407.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,60.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,219.000000,784.000000,873.500000,0.000000,0.000000,1117.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.00000,1.000000,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.000000,0.000000,460.000000,988.000000,1079.000000,0.000000,0.000000,1432.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,0.00000,2.000000,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,78.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,162.000000,752.000000,0.000000,797.500000,1304.000000,1382.500000,676.000000,0.000000,1721.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.00000,2.000000,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,2140.000000,5095.000000,5095.000000,1862.000000,1064.000000,5095.000000,3.000000,2.000000,4.000000,2.000000,6.000000,2.000000,15.000000,4.00000,5.000000,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [204]:
X_all.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'Pav

In [205]:
for col,row in X_all.iteritems():
    print("{}: {} -- na: {}".format(col,row.dtype,row.isnull().sum()))

Id: int64 -- na: 0
MSSubClass: int64 -- na: 0
MSZoning: object -- na: 0
LotFrontage: int64 -- na: 0
LotArea: int64 -- na: 0
Street: object -- na: 0
Alley: object -- na: 0
LotShape: object -- na: 0
LandContour: object -- na: 0
Utilities: object -- na: 0
LotConfig: object -- na: 0
LandSlope: object -- na: 0
Neighborhood: object -- na: 0
Condition1: object -- na: 0
Condition2: object -- na: 0
BldgType: object -- na: 0
HouseStyle: object -- na: 0
OverallQual: int64 -- na: 0
OverallCond: int64 -- na: 0
YearBuilt: int64 -- na: 0
YearRemodAdd: int64 -- na: 0
RoofStyle: object -- na: 0
RoofMatl: object -- na: 0
Exterior1st: object -- na: 0
Exterior2nd: object -- na: 0
MasVnrType: object -- na: 0
MasVnrArea: int64 -- na: 0
ExterQual: object -- na: 0
ExterCond: object -- na: 0
Foundation: object -- na: 0
BsmtQual: object -- na: 0
BsmtCond: object -- na: 0
BsmtExposure: object -- na: 0
BsmtFinType1: object -- na: 0
BsmtFinSF1: int64 -- na: 0
BsmtFinType2: object -- na: 0
BsmtFinSF2: int64 -- na: 

### Set Explanatory Variables as "Categorical Variabes"

In [206]:
X_all['YearRemodAdd'] = X_all['YearRemodAdd'].astype(object)
X_all['YrSold'] = X_all['YrSold'].astype(object)
X_all['MoSold'] = X_all['MoSold'].astype(object)
X_all['YearBuilt'] = X_all['YearBuilt'].astype(object)
X_all['OverallCond'] = X_all['OverallCond'].astype(object)
X_all['OverallQual'] = X_all['OverallQual'].astype(object)
X_all['MSSubClass'] = X_all['MSSubClass'].astype(object)

In [207]:
for col,row in X_all.iteritems():
    print(col,'-----',row.dtype)

Id ----- int64
MSSubClass ----- object
MSZoning ----- object
LotFrontage ----- int64
LotArea ----- int64
Street ----- object
Alley ----- object
LotShape ----- object
LandContour ----- object
Utilities ----- object
LotConfig ----- object
LandSlope ----- object
Neighborhood ----- object
Condition1 ----- object
Condition2 ----- object
BldgType ----- object
HouseStyle ----- object
OverallQual ----- object
OverallCond ----- object
YearBuilt ----- object
YearRemodAdd ----- object
RoofStyle ----- object
RoofMatl ----- object
Exterior1st ----- object
Exterior2nd ----- object
MasVnrType ----- object
MasVnrArea ----- int64
ExterQual ----- object
ExterCond ----- object
Foundation ----- object
BsmtQual ----- object
BsmtCond ----- object
BsmtExposure ----- object
BsmtFinType1 ----- object
BsmtFinSF1 ----- int64
BsmtFinType2 ----- object
BsmtFinSF2 ----- int64
BsmtUnfSF ----- int64
TotalBsmtSF ----- int64
Heating ----- object
HeatingQC ----- object
CentralAir ----- object
Electrical ----- object
1st

### Preprocess Data -- Dummy Variables

In [208]:
def preprocess_features(X):
    df_preprocess = pd.DataFrame(index=X.index) # output dataframe, initially empty
    # iterate through each column
    for colname, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=colname)  
        else: pass
        #join columns to one dataframe
        df_preprocess = df_preprocess.join(col_data)  # collect column(s) in output dataframe
    return df_preprocess

In [209]:
df_= preprocess_features(X_all)  
df_.head()# check everything looks correct

,Id,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C_all,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotFrontage,LotArea,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NAW,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,OverallQual_1,OverallQual_2,OverallQual_3,OverallQual_4,OverallQual_5,OverallQual_6,OverallQual_7,OverallQual_8,OverallQual_9,OverallQual_10,OverallCond_1,OverallCond_2,OverallCond_3,OverallCond_4,OverallCond_5,OverallCond_6,OverallCond_7,OverallCond_8,OverallCond_9,YearBuilt_1879,YearBuilt_1880,YearBuilt_1890,YearBuilt_1895,YearBuilt_1896,YearBuilt_1900,YearBuilt_1901,YearBuilt_1902,YearBuilt_1905,YearBuilt_1907,YearBuilt_1910,YearBuilt_1912,YearBuilt_1914,YearBuilt_1915,YearBuilt_1916,YearBuilt_1917,YearBuilt_1918,YearBuilt_1919,YearBuilt_1920,YearBuilt_1921,YearBuilt_1922,YearBuilt_1923,YearBuilt_1924,YearBuilt_1925,YearBuilt_1926,YearBuilt_1927,YearBuilt_1928,YearBuilt_1929,YearBuilt_1930,YearBuilt_1931,YearBuilt_1932,YearBuilt_1934,YearBuilt_1935,YearBuilt_1936,YearBuilt_1937,YearBuilt_1938,YearBuilt_1939,YearBuilt_1940,YearBuilt_1941,YearBuilt_1942,YearBuilt_1945,YearBuilt_1946,YearBuilt_1947,YearBuilt_1948,YearBuilt_1949,YearBuilt_1950,YearBuilt_1951,YearBuilt_1952,YearBuilt_1953,YearBuilt_1954,YearBuilt_1955,YearBuilt_1956,YearBuilt_1957,YearBuilt_1958,YearBuilt_1959,YearBuilt_1960,YearBuilt_1961,YearBuilt_1962,YearBuilt_1963,YearBuilt_1964,YearBuilt_1965,YearBuilt_1966,YearBuilt_1967,YearBuilt_1968,YearBuilt_1969,YearBuilt_1970,YearBuilt_1971,YearBuilt_1972,YearBuilt_1973,YearBuilt_1974,YearBuilt_1975,YearBuilt_1976,YearBuilt_1977,YearBuilt_1978,YearBuilt_1979,YearBuilt_1980,YearBuilt_1981,YearBuilt_1982,YearBuilt_1983,YearBuilt_1984,YearBuilt_1985,YearBuilt_1986,YearBuilt_1987,YearBuilt_1988,YearBuilt_1989,YearBuilt_1990,YearBuilt_1991,YearBuilt_1992,YearBuilt_1993,YearBuilt_1994,YearBuilt_1995,YearBuilt_1996,YearBuilt_1997,YearBuilt_1998,YearBuilt_1999,YearBuilt_2000,YearBuilt_2001,YearBuilt_2002,YearBuilt_2003,YearBuilt_2004,YearBuilt_2005,YearBuilt_2006,YearBuilt_2007,YearBuilt_2008,YearBuilt_2009,YearBuilt_2010,YearRemodAdd_1950,YearRemodAdd_1951,YearRemodAdd_1952,YearRemodAdd_1953,YearRemodAdd_1954,YearRemodAdd_1955,YearRemodAdd_1956,YearRemodAdd_1957,YearRemodAdd_1958,YearRemodAdd_1959,YearRemodAdd_1960,YearRemodAdd_1961,YearRemodAdd_1962,YearRemodAdd_1963,YearRemodAdd_1964,YearRemodAdd_1965,YearRemodAdd_1966,YearRemodAdd_1967,YearRemodAdd_1968,YearRemodAdd_1969,YearRemodAdd_1970,YearRemodAdd_1971,YearRemodAdd_1972,YearRemodAdd_1973,YearRemodAdd_1974,YearRemodAdd_1975,YearRemodAdd_1976,YearRemodAdd_1977,YearRemodAdd_1978,YearRemodAdd_1979,YearRemodAdd_1980,YearRemodAd

In [212]:
#for col,row in df_.iteritems():
    #print("{}: {} -- na: {}".format(col,row.dtype,row.isnull().sum()))

In [211]:
df_.shape[1]

595

In [94]:
df_.to_csv("Data/test_dummy.csv") #outout csv file

### Split Data into Training/Testing

In [95]:
### Split Dataframe into 70% train 30% testing
from sklearn.model_selection import train_test_split
def test_train_split(X_all,y):
    X_train,X_test,y_train,y_test = train_test_split(X_all,y,test_size=0.30,random_state=42)
    return(X_train,X_test,y_train,y_test)

In [96]:
X_train,X_test,y_train, y_test=test_train_split(X_all,y)

In [97]:
#Training (70%)
training_ = pd.concat([X_train,y_train],axis=1)
training_.to_csv("Data/train_split_dummy.csv")
print(training_.shape[0])

1022


In [98]:
#Testing (30%)
testing_ = pd.concat([X_test,y_test],axis=1)
#Add column "." for predictions in SAS
testing_['prediction'] = "."
testing_.to_csv("Data/test_split_dummy.csv")